In [192]:
from random import *
def puissance (a,e,n):
  p = 1
  while e>0:
    if (e % 2 != 0):
      p=(p*a)%n
    a=(a*a)%n
    e=e//2
  return p
#Commenter tout le code
#taille des messages

In [193]:
def test_premier(n): # test de Fermat

   if ( (puissance(2,n-1,n)==1) and
      (puissance(3,n-1,n)==1) and
      (puissance(5,n-1,n)==1) and
      (puissance(7,n-1,n)==1) and
      (puissance(11,n-1,n)==1) and
      (puissance(13,n-1,n)==1) ):
      return True; # probablement premier (garantie si n<2^15)
   return False

In [194]:
def pgcd(u, v):
  t = 0
  while (v):
    t = u
    u = v
    v = t % v
  if(u<0):
    return -u
  else:
    return u

In [195]:
def bezout(a, b):

  # On sauvegarde les valeurs de a et b.
   a0 = a
   b0 = b

  # On laisse invariant p*a0 + q*b0 = a et  r*a0 + s*b0 = b.
   p = 1  
   q = 0
   r = 0  
   s = 1
   c=1
   quotient=1
   nouveau_r=1
   nouveau_s=1

  # La boucle principale.
   while (b != 0): 
    c = a % b
    quotient = a//b
    a = b
    b = c
    nouveau_r = p - (quotient * r)
    nouveau_s = q - (quotient * s)
    p = r 
    q = s
    r = nouveau_r
    s = nouveau_s
    if p<0:
      p=p+b0
   return p # on n'a besoin que de p #si p est négatif retourne p+phi


In [196]:
#vérifier que le message est plus petit que la clé
def generateurClé():
  c=100
  b=1000
  p=randint(c,b)
  q=randint(c,b)
  e=randint(c,b)
  d=randint(c,b)

  while test_premier(p)==False:
    p = randint(c,b)
  while test_premier(q)==False:
    q = randint(c,b)
  #print("2 nombres probablement premier =",p,q)

  n=p*q
  #print("n=p*q=",n)
  
  phi = (p - 1)*(q - 1)
  #print("phi(n)=",phi)
 
  while 1>=e>=phi or pgcd(e,phi)!=1 :
      e = randint(c,b)
  d= int(bezout(int(e),int(phi)))
  #print("cle publique e=",e)
  #print("cle privée d=", d)
  return e,d,n 

In [197]:
def chiffrer(message,clé,n):
    return puissance(message,clé,n)

In [198]:
def dechiffrer(messageChiffre,clé,n):
    return puissance(messageChiffre,clé,n)

In [199]:
import hashlib

def empreinte(message): 
    m=hashlib.sha256(message)
    return m.digest()
    
a=bytes(1524815)
empreinte(a)

b"E\x90'}=\xc3\xca\x16\xa1n\x14b3\xd1s\x86<\xc0\xb1He]T\xfc\xf8 \xd0nH\xa5s0"

In [200]:
def envoieAuthentifie(message,clePrivee,clePublique):
    #laisser en bytes le message
    #calcule de l'empreinte du message
    e = empreinte(bytes(message))
    print(str(e))
    #Chiffrage de l'empreinte avce la clé privée de l'envoyeur
    c = chiffrer(message,clePrivee[0],clePrivee[1])

    #Concaténation du message et de l'empreinte chiffrée
    e= int.from_bytes(e, byteorder ='little')

    #Taille de l'empreinte
    size= len(str(e))
    concatene = int(str(c)+str(e))

    #Envoie du message finale chiffrée avec la clé publique du receveur
    return chiffrer(concatene,clePublique[0],clePublique[1]), size  
#envoieAuthentifie(1524815)
 

In [203]:
def verif_signature(message,clePrivee,clePublique,tailleEmpreinte):

    #Dechiffrer le message signé
    messageD = str(dechiffrer(message,clePrivee[0],clePrivee[1]))
    
    #Separer le message de l'empreinte
    MessageEmpreinteSep=len(messageD)-tailleEmpreinte
    messageM=int(messageD[0:MessageEmpreinteSep])
    empreinteM=int(messageD[MessageEmpreinteSep:len(messageD)-1])
    print(messageM)
    print(empreinteM)

    #Déchiffrer l'empreinte avec la clé publique
    EmpreinteD = dechiffrer(empreinteM,clePublique[0],clePublique[1])

    #Vérification que l'empreinte déchiffrer correspond à celui du message
    if(int.from_bytes(empreinte(messageM)).equals(EmpreinteD)):
           return True
    else:
            return False
#verif_signature()
    

In [204]:
#Vérifier que le message est plus petit que la clé

#Alice
gA=generateurClé()
cléPubliqueA= [gA[0],gA[2]]
cléPrivéeA= [gA[1],gA[2]]

#CA
gCA=generateurClé()
cléPubliqueCA= [gCA[0],gCA[2]]
cléPrivéeCA= [gCA[1],gCA[2]]

#Envoie message au CA
messageA=envoieAuthentifie(int(str(cléPubliqueA[0])+str(cléPubliqueA[1]))
,cléPrivéeCA,cléPubliqueCA)
print("messageA")
print(messageA)

#Vérifie la signature
#if
verif_signature(messageA[0],cléPrivéeCA,cléPubliqueA,messageA[1])
CAClePubliqeA=chiffrer(cléPubliqueA[0],cléPrivéeCA[0],cléPrivéeCA[1])

#Bob vérifie le certificat d'alice
dechiffrer(CAClePubliqeA,cléPubliqueCA[0],cléPubliqueCA[1])

b'\xf9O\xd3\x01\x05\xd3\xcf0)I\xa8;\xbb\xa2\xc6@HyO\xcc\x1d\x92\xc8F\x8a\xce\xd8i\x9eI\xd6\x1d'
messageA
(17363, 77)


ValueError: invalid literal for int() with base 10: ''

In [ ]:
#BOB
gB=generateurClé()
cléPubliqueB= [gB[0],gB[2]]
cléPrivéeB= [gB[1],gB[2]]

#Choix message taille < 100000
message=1258
messageC=envoieAuthentifie(message,cléPrivéeB,cléPubliqueA)
messageD=verif_signature(messageC,cléPrivéeA,cléPubliqueB,messageC[1])
print("Le message d'origine est")
print(message)
print("Le message crypté est")
print(messageC[0])
print("Le message décrypté est")
print(messageD)

